## Basic Example of one document in Vector store

In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import WebBaseLoader
# from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.embeddings.ollama import OllamaEmbeddings

### Initial Ingest

In [19]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [22]:
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-private",
    embedding=OllamaEmbeddings(),
    persist_directory="./chroma_db"
)
retriever = vectorstore.as_retriever()

### Querying Vector Store

In [23]:
vectorstore.search("Types of Memory","similarity")

[Document(page_content='Memory can be defined as the processes used to acquire, store, retain, and later retrieve information. There are several types of memory in human brains.', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}),
 Document(page_content='Your decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strat

In [27]:
# vectorstore.get()
temp_ids =  ['12937b1f-80df-4af9-bd94-f870e9fa0c19',
  '13166336-6774-4841-9c09-46133b796787',
  '13a9f73b-fe11-46e1-a950-07bd394c9b14',
  '146bc8e3-ff0c-4ea7-a221-93bd28cdcec3',
  '17606735-4be2-466c-bc3b-8aa7365d3f37',
  '1a4a09b5-1047-4bf1-8801-0969898f0479',
  '1ea25c73-5c20-4f70-85db-b7e949dcbef3',
  '1ec4fb21-7ec4-4917-b645-a034d09b9cec',]

In [28]:
for id in temp_ids:
    print(vectorstore.get(id))

{'ids': ['12937b1f-80df-4af9-bd94-f870e9fa0c19'], 'embeddings': None, 'metadatas': [{'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}], 'documents': ['Conversatin samples:\n[\n  {\n    "role": "system",'], 'uris': None, 'data': None}
{'ids': ['13166336-6774-4841-9c09-46133b796787'], 'embeddings': None, 'metadatas': [{'description': 'Building agents with LLM (large language model) as its core controll

### Vectore store from disk - Using chromadb module

In [34]:
import chromadb

In [36]:
persistent_client = chromadb.PersistentClient("./chroma_db")

In [38]:
persistent_client.list_collections()

[Collection(name=langchain), Collection(name=rag-private)]

In [41]:
collection = persistent_client.get_or_create_collection("rag-private")

In [43]:
langchain_chroma = Chroma(
    client=persistent_client,
    collection_name="rag-private",
    embedding_function=OllamaEmbeddings(),
    
)

In [44]:
print("There are", langchain_chroma._collection.count(), "in the collection")

There are 131 in the collection


In [45]:
retriever = langchain_chroma.as_retriever()

In [59]:
langchain_chroma.search("agent-based model","similarity",k=2)

[Document(page_content='pytest\ndataclasses', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}),
 Document(page_content='Case Studies#\nScientific Discovery Agent#\nChemCrow (Bran et al. 2023) is a domain-specific example in which LLM is augmented with 13 expert-designed tools to accomplish tasks across organic synthesis, drug discovery, and materials design. The workflow, implemented in La

### Chain

In [51]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

ollama_llm = "mistral"
model = ChatOllama(model=ollama_llm)

In [52]:
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [53]:
chain.invoke({"question": "What is the agent-based model?"})

' Based on the context provided, an agent-based model refers to a type of autonomous agent system where LLM functions as the agent\'s brain. The agent relies on several key components and can parse user input into tasks, select suitable models from a list to process requests, and help users with various tasks. However, the text does not explicitly define what an agent-based model is in the given context. The term "agent-based model" is not mentioned directly. Instead, it describes the functionality and components of an LLM-powered autonomous agent system.'

In [15]:
chain.invoke({"question": "Can you summarise the document for me?"})

"Of course! Based on the provided context, the answer to your question is:\n\nThe documents describe the potential of Large Language Models (LLMs) as a powerful general problem solver in autonomous agent systems. LLM functions as the agent's brain, complemented by several key components. The documents mention several proof-of-concept demos, such as AutoGPT, GPT-Engineer, and BabyAGI, which serve as inspiring examples of the potential of LLMs. The system comprises of 4 stages: task planning, where LLM works as the brain and parses user requests into multiple tasks; instruction generation, where few-shot examples guide LLM to do task parsing and planning; decision-making, where LLM makes decisions based on the dependencies and arguments associated with each task; and execution, where LLM executes the tasks. The documents also mention HuggingGPT, a system that uses LLM as the brain to perform various tasks."

In [54]:
print(chain.invoke({"question": "How many types of memory does the document talk about?"}))

 Based on the given context, there is no explicit mention of any types of memory discussed in the document. Therefore, I cannot provide an answer to this question with certainty. However, the documents do discuss LLM (large language model) functioning as the brain of an autonomous agent system and several key components that complement it, such as Google Search, Browse Website, Start GPT Agent, Message GPT Agent, List GPT Agents, Delete GPT Agent, Clone Repository, Write to file, Read file, Append to file, Delete file, and Search Files. These can be considered as different functionalities or tools available for the agent system, but they do not necessarily equate to specific types of memory.


In [55]:
print(chain.invoke({"question": "Tell me about Sensory memory?"}))

 {
"role": "assistant",
"thoughts": {
"text": "Sensory memory is a type of memory system that holds sensory information for brief periods, typically less than a second, but potentially up to several seconds. It acts as a buffer that receives and temporarily stores the vast amount of sensory data we receive from our environment through our senses: vision, audition, touch, taste, and smell.",
"reasoning": "The context provided does not directly mention sensory memory. However, I know that in general, an autonomous agent system includes various types of memory systems, one of which is sensory memory. Sensory memory allows the agent to briefly hold incoming sensory data before it is processed further.",
"plan": "- Understand the role and function of sensory memory in an autonomous agent system",
"criticism": "I should ensure that my response accurately reflects the current understanding of sensory memory and its role in an autonomous agent system, based on available information.",
"speak":